# Add costing and emissions calculations to process model

## ROK model code = M5; Shale region: EF-Basin

In [1]:
# simulation case specifications
model_code = 5
case_name = 'EF-Basin'
c_tax_rate = 4.5e-2

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

# Import idaes model serializer to store initialized model
from idaes.core.util import model_serializer as ms

from pyomo.environ import (Constraint,
                           ConstraintList,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Set,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           minimize)

from src.unit_initialization import create_flowsheet, \
                                    define_models, \
                                    define_arcs, \
                                    set_unit_model_variables, \
                                    initialize_flowsheet, \
                                    set_scaling_factors,\
                                    update_model_after_initialization,\
                                    vapor_only_to_vapor_liquid_reformulate,\
                                    update_model_for_optimization, \
                                    unfix_DOFs_pre_optimization, \
                                    fix_DOFs_post_optimization

## Define inlet compositions

In [3]:
M_catalyst = 1167.003367 # kg

In [4]:
import pandas as pd

inlet_df = pd.read_csv('NGL_compositions.csv')

inlet_flow_rate = 481.3888889

dehydro_conv_dict = {'ethane':0.3566,
                     'propane':0.6632,
                     'nbutane':0.5188}

In [5]:
from src.emissions_calculations import calc_lhv_values, calculate_stream_energies, calculate_emissions, create_ghg_objective
from src.costing_function import add_costing,calculate_costs_for_objective
from src.utility_minimization_1d import (
    min_utility,
    PinchDataClass,
    heat_ex_data,
    gen_curves,
    print_HX_results,
    generate_curves,
    heat_data,
    pinch_calc,
    return_data
)

In [6]:
import pandas as pd

In [7]:
cols = ['region','butene', 'pentene', 'hexene', 'heptene', 'octene', 'nonene']
outlet_conc_df = pd.DataFrame(columns=cols)

LHV_cols = ['region','LHV_kJ_mol','MW_fuel','LHV_kJ_s']
LHV_df = pd.DataFrame(columns=LHV_cols)

## Flowsheet initialization with costing for multiple feed compositions

In [8]:
inlet_df = pd.read_csv('NGL_compositions.csv')

In [9]:
cols = ['region','butene','pentene','hexene','heptene','octene','nonene']
outlet_conc_df = pd.DataFrame(columns=cols)

In [10]:
for col in inlet_df.columns:
    if col == 'Species':
        pass
    elif col == 'EF-Basin': # initialization needed only for EF-Basin; EF-1 to EF-12 solved with EF-Basin solution
        region = col
        case_name_current = col
        
        # define flowsheet
        m = create_flowsheet(model_code)
        
        # define unit models
        define_models(m, catalyst_mass = M_catalyst)
        
        # define connections
        define_arcs(m)
        
        # define inlet composition
        inlet_composition_dict = {}

        for i,r in inlet_df.iterrows():
            if r[col] == 0.0:
                inlet_composition_dict[r['Species']] = 1e-6
            else:
                inlet_composition_dict[r['Species']] = round(r[col],4)
        
        ### Define constraints and set-points
        set_unit_model_variables(m, model_code=model_code, feed_flow_rate = inlet_flow_rate, 
                         feed_temp = 308.0, feed_pressure = 700000.0,
                         inlet_composition_dict = inlet_composition_dict,
                         dehydro_conv_dict = dehydro_conv_dict)
        
        ### Scale model components
        if model_code == 2 or model_code == 3:
            set_scaling_factors(m,flow_mol_scaling_factor = 1e-2, inlet_composition_dict = inlet_composition_dict)
        elif model_code == 4 or model_code == 5:
            set_scaling_factors(m,flow_mol_scaling_factor = 1e-3, inlet_composition_dict = inlet_composition_dict)
        else:
            pass
        
        ### Read-in initialization data from .json file with Bakken feed composition
        init_file_name = "./initialization_files/CISTAR_unit_initialization_{}_M{}.json.gz".format('Bakken', model_code)
        ms.from_json(m, fname=init_file_name)
        
        ### Add post-initialization constraints
        update_model_after_initialization(m)
        vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
        vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
        
        ### Read-in flowsheet convergence data from .json file
        init_file_name = "./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name_current, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
        ms.from_json(m, fname=init_file_name)
        
        ### Equipment costing
        add_costing(m)
        ### Heat integration
        # Heat Exchangers and reactors
        min_utility(
            m.fs, [m.fs.H101, m.fs.H103, m.fs.R101], [m.fs.H102, m.fs.H104, m.fs.H105, m.fs.H106, m.fs.R102], 10.0
        )
        m.fs.Qs.fix()
        
        ### Emissions calculations
        calc_lhv_values(m,case_name_current,'./LHV.xlsx','NGL_compositions.csv','NGL_fraction.csv')
        calculate_stream_energies(m)
        calculate_emissions(m,case_name_current,'emissions_factor_by_region.csv')
        create_ghg_objective(m)
        calculate_costs_for_objective(m,c_tax_flag=True, c_tax_val = c_tax_rate)
        
        ### Region-specific square solve for costing initialization
        print('\n ************  Region = {}  ************\n'.format(col))
        DOF_initial = degrees_of_freedom(m)
        print("The final DOF of initialized flowsheet is {0}".format(DOF_initial))
        solver = SolverFactory('ipopt')
        solver.options = {'tol': 1e-6,
                          'bound_push': 1e-8,
                          'max_iter': 500
                         }
        solve_status = solver.solve(m, tee=True)

        print('Min sell price:',m.fs.min_sell_price())
        ms.to_json(m, fname="./initialization_files/CISTAR_solve_with_costing_{}_C_tax_{}_M{}_purge_{}.json.gz".format(case_name_current,m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3)))
        
        conc_list = []
        for col in cols:
            if col == 'region':
                conc_list.append(region)
            else:
                conc_list.append(m.fs.F102.liq_outlet.flow_mol_phase_comp[0.0, 'Liq', col]())

        outlet_conc_df.loc[len(outlet_conc_df.index)] = conc_list
        
        print("====================  Region = {}  ======================".format(region))
        print(outlet_conc_df)
    else:
        pass

Liquid phase flow is zero. Translator block fs.T102 liquid phase composition equations are being modified...

Degenerate constraints removed and liquid phase composition values set to 1e-8

Translator block fs.T102 liquid phase composition equations already modified, no degenerate constraints remaining.


 ************  Region = EF-Basin  ************

Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.co